## Sections
* Mistake-Bounded Learning
* Decision Trees
* Potential Functions and Random Forests
* Generalizations (Error Estimation)
* PAC Learning
* Cross-Validation
 
 ---

## Mistake-Bounded Learning

#### Monotonic Disjunction Basics
* Learner node takes data and returns a prediction (classification).
* Teacher node then responds with whether the prediction was correct or not.
* A learner node is mistake-bound $t$ if for every sequence of challenges, the learner makes at most $t$ mistakes.
* Consider a class in **script C** defined as:
$$C = {\text{Monotonic Disjunctions on n Variables}}$$
$$Domain = \{0, 1\}^n$$
* Where $\{0, 1\}^n$ is a binary string.
* An example of a script C function is:
$$f(x) = x_1 \lor x_3$$
* This function returns 1 every time it is given a bit string that has a 1 in the first or third position. For all other bit strings it returns a 0.
* This function is called a monotonic disjunction because of the $\lor$ (or) operator.

#### General Disjuntion Problem
* Say we are given a script-C function $f \in C$ where $f$ is a monotone disjunction such as the one defined above.
* The learner does not know $f$ and is presented with a bit string $x \in \{0, 1\}^n$. The learner then guesses $f(x)$ and the teacher responds with whether the guess was correct or not.
* To solve this problem for any bit string $x \in \{0, 1\}^n$ in $n$ steps or less, we can start with guessing the following monotonic disjunction:
$$InitialState = f(x) = x_1 \lor x_2 \lor ... \lor x_n$$
* This simple monotonic disjunction is true on every string except for the all 0 string.
* Say the learner receives: 0110010, guesses 1 and it is told it is wrong. The learner now knows that $x_2$, $x_3$, and $x_5$ cannot be in the monotonic disjunction. 
* Following this process, each time a mistake is made the learner can eliminate at least one literal from the monotonic disjunction, thus it will take at maximum $n$ steps to solve the problem.

#### Disjunction Functions
* Consider a function that can also include negations, thus it is not just monotonic.
$$f(x) = \neg x_1 \lor x_2 \lor \neg x_3 \lor x_4 \lor x_5$$
* To apply our learning function from above (the monotonic case) to this more general case we can use feature expansion.
* For an input string of bits of length $n$: 
$$x_1 ... x_n \in \{0, 1\}^n$$
* We expand the string by $2n$ bits by appending to it a new string of bits $y_1 ... y_n$ that is the inverse of the original string.
* For example, the string 0110 (n=4) would be expanded to 01101001 (n=8).
* We can then transform the original disjunction with negated variables into a new function with no negated variables:
$$f(x_1, ..., x_n) = \neg x_1 \lor x_2 \lor \neg x_3 \lor x_4 \lor x_5 \equiv f(x_1, ..., x_n, y_1, ..., y_n) = y_1 \lor x_2 \lor y_3 \lor x_4 \lor x_5$$
* This new algorithm has mistake-bound $2n$.
* **Aside**:
    * On-line learning = learning problem where the learner is fed a stream of examples/data sequentially.
    * Off-line learning = learning problem where the learner is fed a set/batch of examples/data all at once.
* For additional examples of disjunctions problems and on-line learning see: *On-Line Algorithms in Machine Learning* - Avrim Blum.

---

## Decision Trees, Potential Functions, and Random Forests

#### Decision Trees
* Decision trees are Boolean functions (outputs 0 or 1).
* For this example inputs will be a bit string: $x_1 ... x_n \in \{0, 1\}^n$. 
* Size of a decision tree = number of nodes in the tree.
* Depth/height = length of the longest path from the root to a leaf.
* $S$ = training set with $x^m$ input examples and $y^m$ corresponding output examples, $m$ is the number of examples in the training set.
* Question: how to set the root of the decision tree?
    * Define a potential function $\phi(a)$ to determine what criterion we use to put literals at the root.
    * $\phi(a) = min(a, 1-a)$
    * Pick a literal $x_i$, then compute $\phi$ of the probability when we chose an example from $S$ where the outcome $y$ is 0.
* For example, if there are 5 positive examples and 10 negative examples, $\phi Pr(y=0) = \phi \frac{10}{15} = \phi \frac{2}{3} = min(\frac{2}{3}, 1 - \frac{2}{3}) = \frac{1}{3}$.
* $\phi$ gives the error rate for a tree with just one leaf.


#### Potential Functions and Random Forests
* The structure of the tree is determined by your choice of the potential function $\phi$.
    * Another common training function is: $\phi(a) = 2 \cdot a \cdot (1 - a)$
    * This is referred to as the *Gini Function* or *Gini Index*.
* One problem with $\phi(a) = min(a, 1-a)$ is that it is not differentiable (not smooth, graph looks like an upside-down triangle).
    * A differentiable $\phi$ function is a nice feature to have because it is mathematically easier to work with.
* To decide what feature should be at the root:
    * You are looking for the feature with the most "predictive power".
    * Look at feature $x_1$ and see what the error rate will be given $\phi$ if you look at feature $x_1$ first. Then do the same for feature $x_2$ and so on. Ultimately, the feature with the most gain over the trivial tree (lowest error rate) should be picked as the root node (first feature to look at).
* When to stop node selection.
    * 1 When the gain is less than some threshold (very small).
    * 2 Pruning: build an enormous tree and then prune nodes that have the least gain until you get to a desired amount of nodes.
* Random Forests:
    * Build many small decision trees and then take a majority vote of the resulting trees.
    * Algorithm for building many trees:
        * Take your training set $S$ and randomly sub-sample from $S$ to create $S'$, then randomly pick some features from $x_1$ to $x_n$ and create a decision tree from $S'$ using only those features.

---

## Generalizations (Error Estimation)
* Generalization Error = the "true error" of a classifier when subjected to new observations.
* General method for estimating generalization is to take a sample $S$, hold out a subset of the data, and use that subset to estimate the generalization error.
* Another approach is to trade-off model complexity with training error.
    * This can be done using another potential function $\phi$ where $\phi(T) = TrainingError + \alpha \cdot \frac{Size(T)}{|S|}$.
    * Training multiple trees using this heuristic, you choose the tree that minimizes $\phi(T)$.
    * Note that $\phi(T)$ punishes trees that are too large with respect to the training sample size $S$.
* Another approach: MDL = Minimum Description Length Principle
    * The general idea is to take the algorithm that maximally compreses the dataset.
    * If you take just the dataset, the number of bits necessary to compress the dataset is $m \cdot n+1$ where $m$ is the number of examples and $n$ is the number of features +1 for the true outputs $y$.
    * If we build a tree $T$ that is correct on 90% of the dataset, then we can compress the dataset by only storing the 10% of the data that the tree got wrong. 


---

## PAC Learning

* PAC Learning = Probably Approximately Correct Learning
    * Introduced by Valiant (1984).

**PAC Model of Learning**
* Consider a distribution $D$ that is $(0,1)^n$ on the real number line.
* We have a function class $C$ that is a decision tree of size $S$ and a learner $c$ that is an unknown decision tree that we want to learn.
* The learner receives an input $(x, y)$ where $x$ is drawn from $D$ and $y = c(x)$.
    * $y_i$ is always equal to $c(x_i)$ because $c$ is the true function we are trying to learn.
* The goal is to output $h \in C$, a decision method (hypothesis $h$) within the function class $C$ (a decision tree), with the property: $PR_{x~D} [h(x) \neq c(x)] \leq \epsilon$ 
    * When we draw an example $x$ from $D$ we want our guess $h(x)$ to not be incorrect more than $\epsilon$ percent of the time. 
* We also want the learner to be efficient conditional on $n$ (the size of the input) and $S$ the size of the tree. We want a learner that runs on polynomial time conditional on $n$ and $S$.
    * $runtime = polynomial(\frac{1}{\epsilon}, \frac{1}{\delta}, n, S)$
    * As you demand more accuracy (greater $\epsilon$) or you less probability of failure (greater $\delta$) you are allowed to run in more time.
    * $n$ = number of variables.
* When can we PAC learn a function class?
    * Consider an algorithm $A$ that takes a training set and returns a decision tree $T$ that is consistent with $S$ and the size of $T$ is at most size $s$.
    * i.e. $A$ always outputs a consistent hypothesis from script-C given any training set.
    * Given $A$, how can we PAC learn script-C, the original function class?
    * High-level, we draw sufficiently many training points "S" and then use $A$ to find $c \in C$ that is consistent with $S$.
    * The question is, how large should "S" be - another way to approach this question is what is the probability this procedure fails? We want this failure to the $\leq \delta$.
        * The bad outcome is that we output a tree $c$ that is consistent with $S$ but has a true error > $\epsilon$.
        * For example, assume we come up with a tree $c_1$ that has true error > $\epsilon$ (bad), what is the probability that $c_1$ is consistent with $S$?
            * The probability is $\leq (1 - \epsilon)^m$ where $m$ is the number of examples in $S$. This is akin to creating to the probability of drawing a training set $S$ from the distribution $D$ that only has "misleading" examples. 
            * Given that there are at most script-$C$ functions to consider ($c_1, c_2, ... c_n$), the probability of this bad outcome occuring is: $C \cdot (1 - \epsilon)^m$.
        * The above probability is the only bad event that can occur, the probability that we draw any sample which causes us to create a tree $c$ that is consistent with the drawn sample $S$ (the tree works well with examples $S$), but that has a true error rate > $\epsilon$, and our goal is make sure that this probability of a bad outcome is $\leq \delta$: $C \cdot (1 - \epsilon)^m \leq \delta$
        * Using the approximation $(1-x) = e^{-x}$ we can solve for $m$, how many example should be in our training set: $m > \frac{log(\frac{C}{\delta})}{\epsilon}$
    * This function allows us to make the statement: if you choose a number of training samples $m$ that satisfies $m > \frac{log(\frac{C}{\delta})}{\epsilon}$ then with probability $1 - \delta$ ($\delta$ being our accepted failure rate) we will output a tree $c$ that is $1 - \epsilon$ accurate ($\epsilon$ being the error rate of the tree). 
    * One nice property is that we take the log of the possible size of the function class script-C, thus we can have extremely large function classes and still be able to learn them in polynomial time.

**Infinite Function Class**
* PAC-Learning axis-parallel rectangles.
* Given a set of points, labeled + or - based on whether the point is inside or outside the rectange, output $h$ that is $\epsilon$-accurate with probability $1 - \delta$.
* To solve this, we can simply draw the tightest rectangle around the + points, the only question that remains is how large of a training set $S$ to choose.
* A bad outcome for this problem is if a bunch of our sample + points are clustered and the probability of there being positive points outside of our tightest rectangle (the error) is greater than $\epsilon$.

**Half-Spaces**
* Half-space function: $f = SIGN(W \cdot x - \theta)$
    * $W$ is an unknown weight vector and $x$ is a real number, and $\theta$ is a scalar.
    * $f$ is Boolean, the $SIGN$ function outputs 1 if positive, 0 if negative.
    * This can thought of as dividing n-dimensional Euclidean space into two *half-spaces*.
* PAC-Learning algorithms for half-spaces.
    * Given an input $(0, 1, 0, 1, 0, POS)$ we know that $W_2 + W_4 > \theta$.
    * Given more inputs like this we end up with a system of linear inequalities, and linear programming can be used to solve this system of linear inequalities and give us values for $W$.




---

## Cross-Validation
* Method for finding the true error of a classifier.

**Markov's Inequality**
* Let X be a R.V. that takes on only positive values. 
* $Pr[X \geq K \cdot E[X]] \leq \frac{1}{K}$
    * $K$ is a constant and $E[X]$ is the mean of $X$.

**Chebyshev's Inequality**
* Recall that the variance of a R.V. $X$ is $Var[X] = E[(X - E[X])^2]$
    * Variance asks: on average how much does $X$ deviate from its mean, squared?
* Recall $\sqrt{Var[X]}$ is the standard deviation of $X$ = $\sigma$.
* $Pr[|X - \mu| > t \cdot \sigma] \leq \frac{1}{t^2}$
    * $t$ is a constant.
    * This inequality says: when we sample from X, the probability that the given sample deviates more than its expected deviation $\sigma$ by a factor of $t$ is $\leq \frac{1}{t^2}$.

**Chernoff Bound** 
* Say that we have random variables $X_1, X_2, ..., X_n$ and that $E[X_i] = p$.
* $S = \sum_{i=1}^{n} X_i$, the sum of the random variables, and $\mu = E[S] = \sum_{i=1}^{n} E[X_i] = np$.
    * $\mu$ is the expected sum of all the random variables which due to linearity of expectation is equal to the sum of the expected values of each random variable.
* The Chernoff Bound says:
    * $Pr[S > \mu + \delta n] \leq e^{-2n \delta^2}$
        * The probability that a draw $S$ of multiple random variables is bigger than the expected sum $\mu$ by a factor of $\delta$ is $\leq e^{-2n \delta^2}$.
    * Additionally: $Pr[S > \mu - \delta n] \leq e^{-2n \delta \^2}$
    * The above two statements can be combined to say: $Pr[|S - \mu| > \delta n] \leq 2e^{-2n \delta^2}$
        * The probability that the sum of the random variables deviates from the expected sum $\mu$ by a factor of $\delta$ is $\leq 2e^{-2n \delta^2}$.
        * This is akin to saying that the probability that you deviate from $\mu$ is exponentially small in $n$, the sum of many random variables with the same $p$ will not deviate much.
        * Holds for any random variable where we can bound the variance, does not have to be a normal distribution.

**Applying the Chernoff Bound to Estimating the True Error of a Classifier**
* We have a hold-out set $S$ of size $n$.
* Fix $h$ (our tree) that was trained on a training set.
* Recall there is an underlying distribution $D$ from which we draw $S$.
* We are interested in the random variable: $Z = Pr_{x~D} [h(x) \neq c(x)]$
    * Where $c$ is the true function we are trying to learn. $Z$ is the true error of the classifier.
* Let $X_i$ be a random variable that is 1 if $h$ is incorrect on the $i^{th}$ element of S and 0 otherwise.
* $S = \sum_{i=1}^{n} X_i$ and $E[S] = n \cdot p$ where $p$ is the true error rate.
* Say that we set $\delta = 0.1$. Plugging this into the Chernoff Bound we get a right side of:
    * $2e^{-2n \delta^2} = 2e^{-2n \cdot 0.1^2} = 2e^{-0.2n}$
    * Say that we want this "confidence metric" to be less than $\alpha$, how large does $n$ need to be before we satisfy this? Solving for $n$ we get:
        * $n > 50 \cdot log(\frac{\alpha}{2})$
    * $\delta = 0.1$ says I want the error-rate on the hold out set to be 0.1 of the true error rate and $\alpha = 0.1$ says that I want to be 90% confident about this.
    * $\alpha$ says the probability that our error-rate $\delta$ fails to be within 0.1 of the true error rate is at most 0.1.

**Cross-Validation**
* Hold-out is expensive:
    * Data is expensive.
    * If we want to generate multiple methods for generating classifiers, we quickly lose confidence in our estimates. 
* Cross-validation works really well in practice, but there is little theory to explain why it performs so well.
* Cross-validation:
    * Take your entire dataset and break it into folds.
    * First, leave out Fold 1 and train using Folds 2 through Fold K. Then test the classifier on Fold 1.
    * Then hold-out Fold 2 and train on the rest of the classifiers ...
    * This process will occur K times, then you average the error rates of the K classifiers. This will be the true error rate estimation.
* Between 5 and 10 folds is typically used.
* It is very hard to prove anything about cross-validation because re-trains over the folds are not independent.

---